# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
     |████████████████████████████████| 1.1 MB 1.7 MB/s eta 0:00:01
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076096 sha256=93952d06f4c8db1ab4992fc157daed44d5d2ae097eb885cfa15e56553cd334b1
  Stored in directory: /Users/justine.skyler/Library/Caches/pip/wheels/30/bd/87/d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4
Successfully built gmaps
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
#Call the data captured in the WeatherPy
cities_data = pd.read_csv("../WeatherPy/output_data/cities.csv")

cities_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hilo,19.73,-155.09,69.80,77,1,8.05,US,1605376789
1,1,Ushuaia,-54.80,-68.30,39.20,69,75,20.80,AR,1605376789
2,2,Grindavik,63.84,-22.43,33.80,80,40,12.75,IS,1605376789
3,3,Tuatapere,-46.13,167.68,56.79,84,100,5.73,NZ,1605376789
4,4,Asunción Nochixtlán,17.47,-97.23,69.17,38,0,5.75,MX,1605376790
...,...,...,...,...,...,...,...,...,...,...
550,550,Bathsheba,13.22,-59.52,87.80,70,20,11.41,BB,1605376921
551,551,Tautira,-17.73,-149.15,80.60,78,40,6.93,PF,1605376922
552,552,Maragogi,-9.01,-35.22,82.09,71,16,10.60,BR,1605376922
553,553,Bad Freienwalde,52.79,14.03,52.00,83,99,6.76,DE,1605376922


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
gmaps.configure(api_key=g_key)

#Outline the map and details
fig = gmaps.figure(center=(0, 0), zoom_level = 2)
max_intensity = np.max(humidity)

locations = cities_data[['Lat', 'Lng']]
humidity = cities_data['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
#Add a heatlayer to the figure
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
#Limiting the scope by cities with the weather outlined below and put it in a new DataFrame
narrow_city_df = cities_data.loc[(cities_data["Cloudiness"] == 0) & 
                                 (cities_data["Max Temp"] > 70) & 
                                 (cities_data["Max Temp"] < 80) & 
                                 (cities_data["Wind Speed"] < 10)].dropna()

#Print the new DataFream
narrow_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,Salalah,17.02,54.09,78.80,61,0,2.24,OM,1605376816
215,215,Rānāvāv,21.68,69.74,77.43,53,0,4.94,IN,1605376758
272,272,Dingle,11.00,122.67,76.62,91,0,7.14,PH,1605376863
282,282,Odienné,9.51,-7.57,79.20,35,0,2.71,CI,1605376865
351,351,Kidal,18.44,1.41,77.79,18,0,9.95,ML,1605376879
406,406,Atambua,-9.11,124.89,76.01,75,0,1.32,ID,1605376890
422,422,Asarganj,25.15,86.68,71.64,57,0,1.16,IN,1605376748
435,435,Terdāl,16.50,75.05,73.81,61,0,7.70,IN,1605376896
513,513,Teknāf,20.86,92.31,74.23,80,0,5.68,BD,1605376647
536,536,Inhambane,-23.86,35.38,77.00,65,0,8.05,MZ,1605376919


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
#Outlining the information and scope that will be gathered around the above locations
hotel_df = narrow_city_df.loc[:,['City','Country','Lat','Lng']]

hotel_df.insert(4, "Hotel Name", '', allow_duplicates=True)

params = {"keyword": "hotel",
          "radius": 5000,
          "type": "hotel",
          "key": g_key}

In [56]:
#Running a For loop for every index/row in the DataFrame in an attempt to gather hotel information
for index, row in hotel_df.iterrows():
    
    #Populating variables
    lat = row["Lat"]
    long = row["Lng"]
    city_name = row["City"]
    hotel_name = []

    params["location"] = f"{lat},{long}"
    
    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(query_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = name_address['results'][0]['name']
    except IndexError:
        hotel_name.append(np.nan)

In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))